In [1]:
!pip install -qU aif360
!pip install -qU 'aif360[Reductions]'
!pip install -qU 'aif360[inFairness]'

In [2]:
# Load all necessary packages
import sys
#sys.path.insert(1, "../")  
sys.path.insert(1, "work/trendsinai/notebooks/data/GermanDataset")
import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

2024-11-12 05:28:11.865237: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 05:28:11.902936: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 05:28:12.074189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 05:28:12.074248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 05:28:12.075466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [4]:
for el in sys.path:
    print(el)

/home/jovyan/work/trendsinai/notebooks
work/trendsinai/notebooks/data/GermanDataset
/opt/conda/lib/python311.zip
/opt/conda/lib/python3.11
/opt/conda/lib/python3.11/lib-dynload

/opt/conda/lib/python3.11/site-packages


## Step 2 Load dataset, specifying protected attribute, and split dataset into train and test

In Step 2 we load the initial dataset, setting the protected attribute to be age. We then split the original dataset into training and testing datasets. Although we will use only the training dataset in this tutorial, a normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model. 

Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the age attribute. These are key inputs for detecting and mitigating bias, which will be Step 3 and Step 4.

In [26]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

print(type(dataset_orig))
print(dir(dataset_orig))

a = dataset_orig.convert_to_dataframe()
print(a[0].head())

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

<class 'aif360.datasets.german_dataset.GermanDataset'>
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_de_dummy_code_df', '_parse_feature_names', 'align_datasets', 'convert_to_dataframe', 'copy', 'export_dataset', 'favorable_label', 'feature_names', 'features', 'ignore_fields', 'import_dataset', 'instance_names', 'instance_weights', 'label_names', 'labels', 'metadata', 'privileged_protected_attributes', 'protected_attribute_names', 'protected_attributes', 'scores', 'split', 'subset', 'temporarily_ignore', 'unfavorable_label', 'unprivileged_protected_attributes', 'validate_dataset']
   month  credit_amount  inves

In [34]:
print(set(dataset_orig.instance_weights))
for el in set(dataset_orig.instance_weights):
    print(el)

{1.0}
1.0


## Step 3 Compute fairness metric on original training dataset

Now that we've identified the protected attribute 'age' and defined privileged and unprivileged values, we can use aif360 to detect **bias in the dataset**. 

One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter. 

A negative value indicates less favorable outcomes for the unprivileged groups. 

This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class. 

The code below performs this check and displays the output, showing that the difference is -0.169905.

In [27]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.129916


## Step 4 Mitigate bias by transforming the original dataset

The previous step showed that the privileged group was getting 13% more positive outcomes in the training dataset. 

Since this is not desirable, we are going to try to mitigate this bias in the training dataset. As stated above, this is called **pre-processing mitigation** because it happens before the creation of the model.

AI Fairness 360 implements several pre-processing mitigation algorithms. We will choose the **Reweighing algorithm**, which is implemented in the Reweighing class in the aif360.algorithms.preprocessing package. This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

In [28]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

## Step 5 Compute fairness metric on transformed dataset

Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3. 

Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class. 

We see the mitigation step was very effective, the difference in mean outcomes is now 0.0. So we went from a 13% advantage for the privileged group to equality in terms of mean outcome.

In [29]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000
